In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top-doramass/Top 25 K-Drama 2025.csv


Lendo o arquivo que será utilizado para o estudo de caso.

In [54]:
drama = pd.read_csv('/kaggle/input/top-doramass/Top 25 K-Drama 2025.csv')

Inicialmente, teremos uma visão dos dados que estão sendo representados. Abaixo segue os tipos de cada variável(coluna), assim como uma demonstração da tabela.

In [59]:
drama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          25 non-null     object 
 1   Year          25 non-null     int64  
 2   Rating        25 non-null     float64
 3   Watchers      25 non-null     int64  
 4   Episodes      25 non-null     int64  
 5   Ep Duration   25 non-null     object 
 6   Aired On      25 non-null     object 
 7   Genres        25 non-null     object 
 8   Tags          25 non-null     object 
 9   Main Actors   25 non-null     object 
 10  Pop. Ranking  25 non-null     int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 2.3+ KB


In [60]:
drama.head(5)

,Name,Year,Rating,Watchers,Episodes,Ep Duration,Aired On,Genres,Tags,Main Actors,Pop. Ranking
0,Twinkling Watermelon,2023,9.2,98389,16,1:10,"[Monday, Tuesday]","[Romance, Youth, Drama, Fantasy]","[Time Travel, Child From The Future, Sign Lang...","[Ha Eun Gyeol, Ha I Chan, Seol In Ah, Shin Eun...",73
1,Move to Heaven,2021,9.1,112738,10,0:52,[Friday],"[Life, Drama]","[Uncle-Nephew Relationship, Autism, Death, Tea...","[Lee Je Hoon, Tang Jun Sang, Hong Seung Hee]",49
2,Hospital Playlist Season 2,2021,9.1,76300,12,1:40,[Thursday],"[Romance, Life, Drama, Medical]","[Multiple Mains, Band, Music, Multiple Couples...","[Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Ki...",125
3,The Trauma Code: Heroes on Call,2025,9.1,14177,8,0:51,[Friday],"[Action, Comedy, Drama, Medical]","[Hospital Setting, Surgeon Male Lead, Rescue T...","[Ju Ji Hoon, Choo Young Woo, Ha Young, Yoon Gy...",1349
4,Weak Hero Class 1,2022,9.1,101890,8,0:40,[Friday],"[Action, Youth, Drama]","[Teenager Male Lead, Violence, Friendship, Sch...","[Park Ji Hoon, Choi Hyun Wook, Hong Kyung, Kim...",65


Para uma melhor leitura e tratamento dos dados, as strings de genre, aired on, tags e main actors serão divididas em cada termo.

In [56]:
drama['Genres'] = drama['Genres'].astype(str)
drama['Genres'] = drama['Genres'].str.split('|')
drama['Aired On'] = drama ['Aired On'].str.split('|')
drama['Tags'] = drama['Tags'].str.split('|')
drama['Main Actors'] = drama['Main Actors'].str.split('|')

In [58]:
display(drama)

,Name,Year,Rating,Watchers,Episodes,Ep Duration,Aired On,Genres,Tags,Main Actors,Pop. Ranking
0,Twinkling Watermelon,2023,9.2,98389,16,1:10,"[Monday, Tuesday]","[Romance, Youth, Drama, Fantasy]","[Time Travel, Child From The Future, Sign Lang...","[Ha Eun Gyeol, Ha I Chan, Seol In Ah, Shin Eun...",73
1,Move to Heaven,2021,9.1,112738,10,0:52,[Friday],"[Life, Drama]","[Uncle-Nephew Relationship, Autism, Death, Tea...","[Lee Je Hoon, Tang Jun Sang, Hong Seung Hee]",49
2,Hospital Playlist Season 2,2021,9.1,76300,12,1:40,[Thursday],"[Romance, Life, Drama, Medical]","[Multiple Mains, Band, Music, Multiple Couples...","[Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Ki...",125
3,The Trauma Code: Heroes on Call,2025,9.1,14177,8,0:51,[Friday],"[Action, Comedy, Drama, Medical]","[Hospital Setting, Surgeon Male Lead, Rescue T...","[Ju Ji Hoon, Choo Young Woo, Ha Young, Yoon Gy...",1349
4,Weak Hero Class 1,2022,9.1,101890,8,0:40,[Friday],"[Action, Youth, Drama]","[Teenager Male Lead, Violence, Friendship, Sch...","[Park Ji Hoon, Choi Hyun Wook, Hong Kyung, Kim...",65
5,Alchemy of Souls,2022,9.1,128832,20,1:20,"[Saturday, Sunday]","[Action, Historical, Romance, Fantasy]","[Transmigration, Master-Disciple Relationship,...","[Lee Jae Wook, Jung So Min]",33
6,Flower of Evil,2020,9.1,153053,16,1:10,"[Wednesday, Thursday]","[Thriller, Romance, Crime, Melodrama]","[Detective Female Lead, Psychological, Family ...","[Lee Joon Gi, Moon Chae Won, Jang Hee Jin, Seo...",20
7,Moving,2023,9.1,75674,20,0:47,[Wednesday],"[Action, Thriller, Mystery, Supernatural]","[Graphic Violence, Supernatural Power, Multipl...","[Ryu Seung Ryong, Han Hyo Joo, Zo In Sung, Cha...",128
8,Hospital Playlist,2020,9.1,119722,12,1:30,[Thursday],"[Comedy, Life, Drama, Medical]","[Friendship, Multiple Mains, Slow Romance, Mul...","[Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Ki...",39
9,Reply 1988,2015,9.1,123360,20,1:35,"[Friday, Saturday]","[Comedy, Romance, Life, Youth]","[Nostalgia, Multiple Couples, Poor Family, Lov...","[Lee Hye Ri, Go Kyung Pyo, Ryu Jun Yeol, Park ...",36


Vamos dar uma olhada na quantidade de K-Dramas de cada ano que integram a lista dos melhores.

In [62]:
drama['Year'].value_counts()

Year
2023    6
2022    5
2020    4
2021    3
2018    2
2025    1
2015    1
2024    1
2019    1
2017    1
Name: count, dtype: int64